In [2]:
import pandas as pd
import json


In [3]:
test_df = pd.read_csv('test.csv')

In [4]:
test_df = test_df[test_df["label"].isin(["linje1", "linje2", "linje3", "linje4", "linje5", "linje6"])]

In [ ]:
test_df

In [ ]:
drawings = test_df.groupby('task_id').first().reset_index()
drawings

In [8]:
#data loader for the dataset
import torch
from PIL import Image
from torchvision.transforms import functional as F
import os
import cv2

class KBABygglovDataset(torch.utils.data.Dataset):
    def __init__(self, drawings, df,scale = 4):
        self.df = df
        self.drawings = drawings
        self.transforms = None
        self.scale = scale
        #self.task_ids = df["task_id"].unique()
    
    def __len__(self):
        return len(self.drawings)
    
    def __getitem__(self, idx):
        task_id = self.drawings.iloc[idx]["task_id"]
        rows = self.df[self.df["task_id"] == task_id]
        image = cv2.imread(rows.iloc[0]["image"])
        #contour = find_bounding_box(image)
        
        
        # Normalize bounding box coordinates
        width, height = rows.iloc[0]["original_width"], rows.iloc[0]["original_height"]
        target = {}
        boxes = []
        labels = []

        for i, row in rows.iterrows():
            x1 = row['x'] * row["original_width"] //100 
            y1 = row['y'] * row["original_height"] //100
            width = row['width'] * row["original_width"] //100
            height = row['height'] * row["original_height"] //100
            if width == 0 or height == 0:
                print("FELLLLLL", task_id)
                print(x1, y1, width, height)
                print(row)
            boxes.append([x1, y1, x1 + width, y1 + height])
            labels.append(1)
        target["boxes"] = torch.as_tensor(boxes, 
                                dtype = torch.float32)
        target["labels"]=torch.as_tensor(labels,
                        dtype = torch.int64)
        target["image_id"] = torch.as_tensor([task_id])
        #scale the image and target to half size
        image = cv2.resize(image, (image.shape[1]// self.scale,  image.shape[0]// self.scale))

        target["boxes"] = target["boxes"]/ self.scale

        image = F.to_tensor(image)
        #cropped_image, target = crop_to_contour(image, target, contour)
        return image, target





test_dataset = KBABygglovDataset(drawings, test_df)

In [9]:
# Collate image-target pairs into a tuple.
def collate_fn(batch):
    return tuple(zip(*batch))
# Create the DataLoaders from the Datasets. 

test_dl = torch.utils.data.DataLoader(test_dataset, 
                               batch_size = 1, 
                              shuffle = False, 
                      collate_fn = collate_fn)

In [10]:
import torch

In [19]:
model = torch.load('distance_marker.model')

In [20]:
def unbatch(batch, device):
    """
    Unbatches a batch of data from the Dataloader.
    Inputs
        batch: tuple
            Tuple containing a batch from the Dataloader.
        device: str
            Indicates which device (CPU/GPU) to use.
    Returns
        X: list
            List of images.
        y: list
            List of dictionaries.
    """
    X, y = batch
    X = [x.to(device) for x in X]
    y = [{k: v.to(device) for k, v in t.items()} for t in y]
    return X, y
def train_batch(batch, model, optimizer, device):
    """
    Uses back propagation to train a model.
    Inputs
        batch: tuple
            Tuple containing a batch from the Dataloader.
        model: torch model
        optimizer: torch optimizer
        device: str
            Indicates which device (CPU/GPU) to use.
    Returns
        loss: float
            Sum of the batch losses.
        losses: dict
            Dictionary containing the individual losses.
    """
    model.train()
    X, y = unbatch(batch, device = device)
    optimizer.zero_grad()
    losses = model(X, y)
    loss = sum(loss for loss in losses.values())
    loss.backward()
    optimizer.step()
    return loss, losses
@torch.no_grad()
def validate_batch(batch, model, optimizer, device):
    """
    Evaluates a model's loss value using validation data.
    Inputs
        batch: tuple
            Tuple containing a batch from the Dataloader.
        model: torch model
        optimizer: torch optimizer
        device: str
            Indicates which device (CPU/GPU) to use.
    Returns
        loss: float
            Sum of the batch losses.
        losses: dict
            Dictionary containing the individual losses.
    """
    model.train()
    X, y = unbatch(batch, device = device)
    optimizer.zero_grad()
    losses = model(X, y)
    loss = sum(loss for loss in losses.values())
    return loss, losses

In [21]:
@torch.no_grad()
def predict_batch(batch, model, device):
    model.to(device)
    model.eval()
    X, _ = unbatch(batch, device = device)
    predictions = model(X)
    return [x.cpu() for x in X], predictions
def predict(model, data_loader, device = "cpu"):
    images = []
    predictions = []
    for i, batch in enumerate(data_loader):
        X, p = predict_batch(batch, model, device)
        images.append(X)
        predictions.append(p)
    
    return images, predictions

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
images, predictions = predict(model, test_dl, device = device)

In [ ]:
device

In [24]:
import torchvision
def decode_prediction(prediction, 
                      score_threshold = 0.8, 
                      nms_iou_threshold = 0.2):
    """
    Inputs
        prediction: dict
        score_threshold: float
        nms_iou_threshold: float
    Returns
        prediction: tuple
    """
    boxes = prediction["boxes"]
    scores = prediction["scores"]
    labels = prediction["labels"]
    # Remove any low-score predictions.
    if score_threshold is not None:
        want = scores > score_threshold
        boxes = boxes[want]
        scores = scores[want]
        labels = labels[want]
    # Remove any overlapping bounding boxes using NMS.
    if nms_iou_threshold is not None:
        want = torchvision.ops.nms(boxes = boxes, scores = scores, 
                                iou_threshold = nms_iou_threshold)
        boxes = boxes[want]
        scores = scores[want]
        labels = labels[want]
    return {"boxes": boxes, 
            "labels": labels, 
            "scores": scores}

In [25]:
from Object_detection.object_detection_helper import visualize_image_with_bounding_boxes_colored_img

In [ ]:
for i in range(len(images)):
    print(i)
    new_pred = decode_prediction(predictions[i][0], score_threshold = 0.6)
    print(new_pred)
    visualize_image_with_bounding_boxes_colored_img(images[i][0], new_pred)